In [ ]:
import json
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

file_name = 'talk_tags.json'

from google.colab import files
#uploaded = files.upload()  # Это позволит загрузить файл через интерфейс Google Colab

file_name = 'talk_tags.json'

# Чтение JSON файла
with open(file_name, 'r') as f:
    data = json.load(f)

# Преобразуем JSON данные в DataFrame для удобной работы
df = pd.json_normalize(data)

# Проверим первые несколько строк данных
df.head(6)

type(df['text'][0])


df = pd.DataFrame(data)

    # Преобразование всех элементов колонки text из списка в строку
df['text'] = df['text'].apply(lambda x: ','.join(map(str, x)))

type(df['text'][0])

# Инициализация инструментов NLTK
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


lemmatizer = WordNetLemmatizer()



# Функция предобработки текста
def preprocess_text(text):
    # Приведение к нижнему регистру
    text = text.lower()

    text = text.replace(',', ' ')

    # Удаление лишних символов, которые не являются словами
    text = re.sub(r'[^\w\s]', '', text)  # Удаляем пунктуацию, кроме слов и пробелов

    # Токенизация текста
    tokens = word_tokenize(text)

    # Лемматизация
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Удаление пустых строк и коротких токенов
    clean_tokens = [token for token in lemmatized_tokens if len(token) > 1]

    return ' '.join(clean_tokens)


import nltk
nltk.download('punkt_tab')
df['cleaned_text'] = df['text'].apply(preprocess_text)

df['cleaned_text']

# Список слов, которые нужно удалить
words_to_remove = ['ctxt', 'str', 'text']

# Функция для удаления лишних слов
def remove_unwanted_words(text):
    # Разбиваем текст на слова и фильтруем
    clean_tokens = [word for word in text.split() if word not in words_to_remove]
    return ' '.join(clean_tokens)

# Применяем функцию к колонке 'cleaned_text'
df['cleaned_text_after'] = df['cleaned_text'].apply(remove_unwanted_words)

# Вывод первых строк
print(df['cleaned_text_after'].head())

# Список слов, которые нужно удалить
words_to_remove = ['ctxt', 'str', 'text']

# Функция для удаления лишних слов
def remove_unwanted_words(text):
    # Разбиваем текст на слова и фильтруем
    clean_tokens = [word for word in text.split() if word not in words_to_remove]
    return ' '.join(clean_tokens)

# Применяем функцию к колонке 'cleaned_text'
df['cleaned_text_after'] = df['cleaned_text'].apply(remove_unwanted_words)

# Вывод первых строк
print(df['cleaned_text_after'].head())

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from collections import Counter
import numpy as np


# Токенизация: разбиваем текст на слова
def tokenize(text):
    return text.split()

# Создание словаря (включает специальные токены)
def build_vocab(texts, min_freq=1):
    counter = Counter()
    for text in texts:
        counter.update(tokenize(text))

    # Фильтруем редкие слова
    vocab = {word: idx + 2 for idx, (word, count) in enumerate(counter.items()) if count >= min_freq}
    vocab['<PAD>'] = 0
    vocab['<UNK>'] = 1
    return vocab

# Пример создания словаря
texts = df['cleaned_text_after'].tolist()
vocab = build_vocab(texts)
print("Размер словаря:", len(vocab))


# Функция преобразования текста в индексы
def text_to_indices(text, vocab):
    return [vocab.get(word, vocab['<UNK>']) for word in tokenize(text)]

# Преобразуем все тексты в индексы
df['text_indices'] = df['cleaned_text_after'].apply(lambda x: text_to_indices(x, vocab))

# Пример
print(df['text_indices'].head())


from torch.nn.utils.rnn import pad_sequence

# Преобразуем тексты в тензоры с паддингом
def collate_fn(batch):
    batch = [torch.tensor(item) for item in batch]
    return pad_sequence(batch, batch_first=True, padding_value=0)

# Делим данные на train, val, test
train_texts, test_texts = train_test_split(df['text_indices'].tolist(), test_size=0.2, random_state=42)

train_loader = DataLoader(train_texts, batch_size=32, collate_fn=collate_fn, shuffle=True)
test_loader = DataLoader(test_texts, batch_size=32, collate_fn=collate_fn)

print("Размер тренировочного набора:", len(train_loader.dataset))
print("Размер тестового набора:", len(test_loader.dataset))


import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import Transformer
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# Гиперпараметры
vocab_size = len(vocab)  # Размер словаря
embed_dim = 128          # Размерность эмбеддингов
nhead = 4                # Количество "голов" в multi-head attention
hidden_dim = 256         # Размер скрытых слоёв (feed-forward)
num_layers = 2           # Количество энкодеров в Transformer
dropout = 0.1            # Дропаут
lr = 0.001               # Скорость обучения
num_epochs = 10          # Количество эпох


class TransformerModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, nhead, hidden_dim, num_layers, dropout):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.pos_encoder = PositionalEncoding(embed_dim, dropout)
        self.transformer = Transformer(
            d_model=embed_dim,
            nhead=nhead,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=hidden_dim,
            dropout=dropout,
            batch_first=True
        )
        self.fc_out = nn.Linear(embed_dim, vocab_size)

    def forward(self, src, tgt):
        src = self.embedding(src)  # Входной текст
        tgt = self.embedding(tgt)  # Текст, который нужно предсказать
        src = self.pos_encoder(src)
        tgt = self.pos_encoder(tgt)
        output = self.transformer(src, tgt)
        output = self.fc_out(output)  # Прогноз следующего токена
        return output

# Позиционное кодирование для добавления информации о порядке слов
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

# Сдвигаем текст для обучения
def create_src_tgt_pairs(data, seq_len=10):
    src = []
    tgt = []
    for sentence in data:
        for i in range(1, len(sentence)):
            src.append(sentence[:i])
            tgt.append(sentence[1:i+1])
    return src, tgt

# Преобразуем данные
data = df['text_indices'].tolist()
src_data, tgt_data = create_src_tgt_pairs(data)

# Паддинг и DataLoader
def collate_fn(batch):
    src_batch, tgt_batch = zip(*batch)
    src_batch = pad_sequence([torch.tensor(x) for x in src_batch], batch_first=True, padding_value=0)
    tgt_batch = pad_sequence([torch.tensor(x) for x in tgt_batch], batch_first=True, padding_value=0)
    return src_batch, tgt_batch

train_data, test_data = train_test_split(list(zip(src_data, tgt_data)), test_size=0.2, random_state=42)
train_loader = DataLoader(train_data, batch_size=32, collate_fn=collate_fn)
test_loader = DataLoader(test_data, batch_size=32, collate_fn=collate_fn)


model = TransformerModel(vocab_size, embed_dim, nhead, hidden_dim, num_layers, dropout)
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=lr)

# Цикл обучения
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for src, tgt in train_loader:
        optimizer.zero_grad()
        output = model(src, tgt[:, :-1])  # Сдвигаем на 1 для предсказания
        loss = criterion(output.reshape(-1, vocab_size), tgt[:, 1:].reshape(-1))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader):.4f}")


torch.save(model.state_dict(), "model_weights.pth")


# Добавьте '<unk>' в словарь с индексом 0
vocab = {word: idx for idx, word in enumerate(vocab)}  # Используйте свой список слов
vocab["<unk>"] = 0  # Добавляем токен для неизвестных слов

# Обратный словарь
reverse_vocab = {idx: word for word, idx in vocab.items()}

# Токенизатор (преобразует строку в индексы)
def tokenizer(text):
    return [vocab.get(word, vocab["<unk>"]) for word in text.split()]

# Детокенизатор (преобразует индексы обратно в строку)
def detokenizer(indices):
    return " ".join([reverse_vocab.get(idx, "<unk>") for idx in indices])

def generate_text_top_p(model, start_sequence, max_len=50, temperature=1.0, top_p=0.9):
    model.eval()
    tokens = tokenizer(start_sequence)  # Токенизируем начальную последовательность
    input_seq = torch.tensor(tokens).unsqueeze(0)  # Добавляем размер batch

    generated = tokens.copy()  # Сохраняем начальную последовательность

    for _ in range(max_len):
        # Получаем предсказание модели
        with torch.no_grad():
            output = model(input_seq, input_seq)  # Вход и цель одинаковы при генерации
            next_token_logits = output[0, -1, :]  # Последний токен

        # Применяем температурное масштабирование
        next_token_logits = next_token_logits / temperature

        # Применяем top-p сэмплинг
        sorted_logits, sorted_indices = torch.sort(next_token_logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
        cutoff = torch.where(cumulative_probs > top_p)[0][0]
        filtered_logits = sorted_logits[:cutoff + 1]
        filtered_indices = sorted_indices[:cutoff + 1]

        probs = F.softmax(filtered_logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1).item()

        # Добавляем предсказанный токен к последовательности
        generated.append(filtered_indices[next_token].item())
        input_seq = torch.tensor(generated).unsqueeze(0)

        # Останавливаем генерацию при достижении специального токена (если есть)
        if filtered_indices[next_token].item() == vocab.get('<eos>', -1):
            break

    # Декодируем сгенерированные токены в текст
    return detokenizer(generated)

# Пример генерации с использованием top-p
start_sequence = "adventurer"
generated_text = generate_text_top_p(model, start_sequence, max_len=30, temperature=0.7, top_p=0.9)
print("Generated Text:", generated_text)


# Сохранение словаря
with open("vocab.json", "w") as f:
    json.dump(vocab, f)